In [2]:
!pip install keras

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 377 kB 835 kB/s eta 0:00:01     |██████████████████████████      | 307 kB 835 kB/s eta 0:00:01


In [1]:
from keras.layers import Dense, Input, LSTM, Embedding, BatchNormalization,Bidirectional
from keras.layers import *
import os
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd
import numpy as np
import keras
import h5py
from keras.optimizers import RMSprop

# Position_Embedding
#! -*- coding: utf-8 -*-
#%%

class Position_Embedding(Layer):
 
    def __init__(self, size=None, mode='sum', **kwargs):
        self.size = size #必须为偶数
        self.mode = mode
        super(Position_Embedding, self).__init__(**kwargs)
 
    def call(self, x):
        if (self.size == None) or (self.mode == 'sum'):
            self.size = int(x.shape[-1])
        batch_size,seq_len = K.shape(x)[0],K.shape(x)[1]
        position_j = 1. / K.pow(10000., \
                                 2 * K.arange(self.size / 2, dtype='float32' \
                               ) / self.size)
        position_j = K.expand_dims(position_j, 0)
        position_i = K.cumsum(K.ones_like(x[:,:,0]), 1)-1 #K.arange不支持变长，只好用这种方法生成
        position_i = K.expand_dims(position_i, 2)
        position_ij = K.dot(position_i, position_j)
        position_ij = K.concatenate([K.cos(position_ij), K.sin(position_ij)], 2)
        if self.mode == 'sum':
            return position_ij + x
        elif self.mode == 'concat':
            return K.concatenate([position_ij, x], 2)
 
    def compute_output_shape(self, input_shape):
        if self.mode == 'sum':
            return input_shape
        elif self.mode == 'concat':
            return (input_shape[0], input_shape[1], input_shape[2]+self.size)

def buid_model():
    # LSTM 模型
    print('lstm model start...\n')

    # 标题输入：接收一个含有 200 个整数的序列，每个整数在 1 到 3812202 之间。
    main_input1 = Input(shape=(200,), name='main_input1', dtype='int32')
    with h5py.File('../../get_w2v_feat/w2v1_pre_ad/size_32_w100_count_0/embeddings_matrix_ad_size_32_w100_count_0.h5','r') as f:
        embeddings_matrix1 = np.array(f.get('embeddings_matrix'))
    embeddings1 = Embedding(input_dim = len(embeddings_matrix1), # 字典长度
                                output_dim = 32, # 词向量 长度（100）
                                weights=[embeddings_matrix1], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(main_input1)
    
    main_input2 = Input(shape=(200,), name='main_input2', dtype='int32')
    with h5py.File('../../get_w2v_feat/w2v1_pre_advertiser/embeddings_matrix_advertiser_size_32_w100_count_0.h5','r') as f:
        embeddings_matrix2 = np.array(f.get('embeddings_matrix'))
    embeddings2 = Embedding(input_dim = len(embeddings_matrix2), # 字典长度
                                output_dim = 32, # 词向量 长度（100）
                                weights=[embeddings_matrix2], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(main_input2)
    
    main_input3 = Input(shape=(200,), name='main_input3', dtype='int32')
    with h5py.File('../../get_w2v_feat/w2v1_pre_creative_id/embeddings_matrix_creative_size_32_w100_count_0.h5','r') as f:
        embeddings_matrix3 = np.array(f.get('embeddings_matrix'))
    embeddings3 = Embedding(input_dim = len(embeddings_matrix3), # 字典长度
                                output_dim = 32, # 词向量 长度（100）
                                weights=[embeddings_matrix3], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(main_input3)
    
    emb = keras.layers.concatenate([embeddings1, embeddings2, embeddings3])
    
    lstm_out = Bidirectional(LSTM(100,activation='softsign',return_sequences = True))(emb)
    lstm_out = Bidirectional(LSTM(100,activation='softsign'))(lstm_out)
    
    # 堆叠全连接
#     den_x = Dense(32, activation='relu')(lstm_out)
    
#     # 辅助输入数据
#     auxiliary_input = Input(shape=(48,), name='aux_input')
#     # 将辅助输入与lstm输出连接起来
    
#     lstm_out = Dense(30, activation='relu')(lstm_out)
#     lstm_out = BatchNormalization()(lstm_out)
    # 辅助损失函数
#     auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)
    # 主输出
    main_output = Dense(10, activation='softmax', name='main_output')(lstm_out)

    # 定义一个具有两个输入输出的模型
    model = keras.models.Model(inputs=[main_input1,main_input2,main_input3],
                               outputs=[main_output])  # 这里的输入输出顺序与fit时一致就好
    
    opt = RMSprop(lr=0.01,  clipnorm=1.0)
    model.compile(optimizer=opt,
                  loss={'main_output': 'categorical_crossentropy'},
                 metrics=['accuracy'])

    print('lstm model geted...\n')
    return model


def data_load():
    print('loading data ... \n')

    with h5py.File('../../get_w2v_feat/w2v1_pre_ad/size_32_w100_count_0/word_train_ad_w2v.h5', 'r') as f:
        data = np.array(f.get('word_data'))
        
    label = pd.read_csv('../../train_preliminary/user.csv').sort_values(by=['user_id'])

    train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.2, random_state=2020)

    train_y_age = train_y['age'].values - 1
    train_y_age = keras.utils.np_utils.to_categorical(train_y_age, num_classes=10)
    train_y_gender = train_y['gender'].values - 1

    test_y_age = test_y['age'].values - 1
    test_y_age = keras.utils.np_utils.to_categorical(test_y_age, num_classes=10)
    test_y_gender = test_y['gender'].values - 1

    print('get data ... \n')

    return train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender

def load_data2():
    
    with h5py.File('../../get_w2v_feat/w2v1_pre_advertiser/word_train_advertiser_w2v_w100.h5', 'r') as f:
        data = np.array(f.get('word_data'))

    train_x, test_x= train_test_split(data, test_size=0.2, random_state=2020)
    return train_x, test_x

def load_data3():
    
    with h5py.File('../../get_w2v_feat/w2v1_pre_creative_id/word_train_creative_w2v_w100.h5', 'r') as f:
        data = np.array(f.get('word_data'))

    train_x, test_x= train_test_split(data, test_size=0.2, random_state=2020)
    return train_x, test_x

Using TensorFlow backend.


In [2]:
model = buid_model()

lstm model start...







lstm model geted...



In [3]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input2 (InputLayer)        (None, 200)          0                                            
__________________________________________________________________________________________________
main_input1 (InputLayer)        (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 200, 32)      1851872     main_input2[0][0]                
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 32)      96875552    main_input1[0][0]                
__________________________________________________________________________________________________
position__

In [4]:
train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender = data_load()

train_x2, test_x2 = load_data2()

train_x3, test_x3 = load_data3()

loading data ... 

get data ... 



In [ ]:

def get_filename_for_saving(save_dir):
    return os.path.join(save_dir,
                        "lstm100_2_comb_age_w2v_pre{val_loss:.3f}-{val_acc:.3f}-{epoch:03d}-{loss:.3f}-{acc:.3f}.hdf5")


print('lstm model fit...\n')
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=get_filename_for_saving(''),
    save_best_only=False)
stopping = keras.callbacks.EarlyStopping(patience=8)
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.0001)

model.fit({'main_input1': train_x ,'main_input2': train_x2,'main_input3': train_x3},
          {'main_output': train_y_age},
          epochs=100,
          batch_size=256,
          validation_data=({'main_input1': test_x,'main_input2': test_x2,'main_input3': test_x3},
                           {'main_output': test_y_age}),
          callbacks=[checkpointer, reduce_lr, stopping])

lstm model fit...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_util.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 109208736 elements. This may consume a large amount of memory.
  num_elements)


Train on 720000 samples, validate on 180000 samples
Epoch 1/100
193280/720000 [=======>......................] - ETA: 2:07:55 - loss: 1.5805 - acc: 0.3586

In [ ]:



# pre = model.predict(test_x,verbose=1)

# #评估结果
# from sklearn.metrics import confusion_matrix, classification_report
# y_ = np.reshape(np.argmax(test_y,axis=1),[-1])
# pre_ = np.reshape(np.argmax(pre, axis=1),[-1])
# #每个类的各项指标
# cm = confusion_matrix(y_, pre_)
# # np.set_printoptions(precision=3)
# cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
# print(cm_normalized)
# print(classification_report(y_, pre_))